In [24]:
# Imported libraries
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn import *
import sklearn.svm as svm
import sklearn.preprocessing as prep
import sklearn.model_selection as ms
import sklearn.metrics as metrics
from tqdm.notebook import tqdm
import sklearn.linear_model as lm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

### Part a

In [25]:
# Definition of function to read data
def read_data(text_file):
    lines = []
    with open(text_file, "r") as data_file:
        data = data_file.read()
        lines = data.split("\n")
    lines = lines[:-1]

    y_label = []
    vals , R , C = [] , [] , []
    max_col_len = 0
    for index, line in enumerate(lines):
        line_data = line.split(" ")
        y_label.append(int(line_data[0]))
        for x in line_data[1:]:
            col, val = x.split(":")
            col, val = int(col), int(val)
            R.append(index)
            C.append(col)
            vals.append(val)
            if max_col_len < col:
                max_col_len = col
    sparse_matrix = csr_matrix((vals, (R, C)), shape=(len(lines), max_col_len+1))
    return sparse_matrix, np.array(y_label)

# Calling read_data function to read Data_Q1 file (part a))
X_data, y_labels = read_data("Data_Q1.txt")
print(" Dimension of X matrix ", X_data.shape )
print(" Dimension of y matrix ", y_labels.shape)



 Dimension of X matrix  (4143, 54878)
 Dimension of y matrix  (4143,)


### Part b

In [26]:
# Number of points in each class
cl , number = np.unique(y_labels, return_counts=True)
print("Class :", cl[0], " - ", number[0])
print("Class :" , cl[1], " - ", number[1])

Class : -1  -  1933
Class : 1  -  2210


###
 Number of samples for class 1 and -1 don't differ so much. since there is not class imbalance issues. While splitting data, we must ensure that the distribution of the data is the same for each class.



### Part c

In [27]:
#Splitting data  - Train : Test  = 8 : 2
T1, T2, Y1, Y2 = ms.train_test_split(X_data, y_labels, random_state=12, test_size=0.2, stratify=y_labels)
print("Train X_Data dimension: ", T1.shape)
print(" Test X Data dimension: ", T2.shape)

print("Train Y-labels dimension: ", Y1.shape)
print(" Test Y- labels dimension: ", Y2.shape)

Train X_Data dimension:  (3314, 54878)
 Test X Data dimension:  (829, 54878)
Train Y-labels dimension:  (3314,)
 Test Y- labels dimension:  (829,)


In [28]:
# Checking if the class wise distribtion is same in Test and Train data
c1, col1 = np.unique(Y1, return_counts=True)
c2, col2 = np.unique(Y2, return_counts=True)

print("Class : ", c1[0])
print(" Training data " , col1[0] )
print(" Test data " , col2[0])
print(" --------------------------------------------")
print("Class : ", c1[1])
print( " Training data " , col1[1] )
print( " Test data " , col2[1])

print(" ------------------------------------------- ")
print("Ratio of data in class -1 : data in class 1")

print("Training" , col1[0]/col1[1] )
print("Test" , col2[0]/col2[1])



Class :  -1
 Training data  1546
 Test data  387
 --------------------------------------------
Class :  1
 Training data  1768
 Test data  442
 ------------------------------------------- 
Ratio of data in class -1 : data in class 1
Training 0.8744343891402715
Test 0.8755656108597285



### There is no class imbalance issue in train and test data also. they are similar

### Ranges of the chosen tuning parameters

C is chosen to begin at [0.5,1.5]

This parameter set handles both increased and decreased regularisation.

gamma is choosen to start from  [0.1,10]

We have a good range to work with with this range, thus gamma can be little and large.


n_neighbors is choosen to start from  [1,10]

A broader range will lengthen the inference period.

n_estimators is choosen to start from  [50,500]

min_weight_fraction_leaf is choosen to have range  [0,0.5],

Only this range is acceptable for this characteristic.

In [29]:
best_models_list = []

# Logistic regression with L2 regularizer (Hyperparameter: regularization constant C)

model_1 = lm.LogisticRegression(penalty="l2", solver="lbfgs", n_jobs=-1)
tuning = {"C": np.linspace(1e-10, 1, num=1)}
print("Model Description: ", model_1)
grid_cv1 = ms.GridSearchCV(model_1, tuning, scoring="accuracy", refit=True, cv=4, verbose=2, n_jobs=-1)
grid_cv1.fit(T1, Y1)
best_models_list.append(grid_cv1.best_estimator_)
print("---------------------------------------------")
print("Best Parameter obtained: ", tuning) 
print("Best Score obtained: ", grid_cv1.best_score_)

Model Description:  LogisticRegression(n_jobs=-1)
Fitting 4 folds for each of 1 candidates, totalling 4 fits
---------------------------------------------
Best Parameter obtained:  {'C': array([1.e-10])}
Best Score obtained:  0.5334944610525457


In [30]:
# Logistic regression with L1 regularizer (Hyperparameter: regularization constant C)

model_2 = lm.LogisticRegression(penalty="l1", solver="saga", n_jobs=-1)
tuning = {"C": np.linspace(1e-10, 1, num=1)}
print("Model Description: ", model_2)
grid_cv2 = ms.GridSearchCV(model_2, tuning, scoring="accuracy", refit=True, cv=4, verbose=2, n_jobs=-1)
grid_cv2.fit(T1, Y1)
best_models_list.append(grid_cv2.best_estimator_)
print("---------------------------------------------")
print("Best Parameter obtained: ", tuning) 
print("Best Score obtained: ", grid_cv2.best_score_)

Model Description:  LogisticRegression(n_jobs=-1, penalty='l1', solver='saga')
Fitting 4 folds for each of 1 candidates, totalling 4 fits
---------------------------------------------
Best Parameter obtained:  {'C': array([1.e-10])}
Best Score obtained:  0.5334944610525457


In [31]:
# Soft-margin SVM with L2 regularizer (Hyperparameter: regularization constant C)
model_3 = svm.LinearSVC(penalty="l2")
tuning = {"C": np.linspace(1e-10, 1, num=1)}
print("Model Description: ", model_3)
grid_cv3 = ms.GridSearchCV(model_3, tuning, scoring="accuracy", refit=True, cv=4, verbose=2, n_jobs=-1)
grid_cv3.fit(T1, Y1)
best_models_list.append(grid_cv3.best_estimator_)
print("---------------------------------------------")
print("Best Parameter obtained: ", tuning) 
print("Best Score obtained: ", grid_cv3.best_score_)

Model Description:  LinearSVC()
Fitting 4 folds for each of 1 candidates, totalling 4 fits
---------------------------------------------
Best Parameter obtained:  {'C': array([1.e-10])}
Best Score obtained:  0.7474355780493349


In [32]:
# Soft-margin SVM with L1 regularizer (Hyperparameter: regularization constant C)
model_4 = svm.LinearSVC(penalty="l1", dual=False)
tuning =  {"C": np.linspace(1e-10, 1, num=1)}
print("Model Description: ", model_4)
grid_cv4 = ms.GridSearchCV(model_4, tuning, scoring="accuracy", refit=True, cv=4, verbose=2, n_jobs=-1)
grid_cv4.fit(T1, Y1)
best_models_list.append(grid_cv4.best_estimator_)
print("---------------------------------------------")
print("Best Parameter obtained: ", tuning) 
print("Best Score obtained: ", grid_cv4.best_score_)

Model Description:  LinearSVC(dual=False, penalty='l1')
Fitting 4 folds for each of 1 candidates, totalling 4 fits
---------------------------------------------
Best Parameter obtained:  {'C': array([1.e-10])}
Best Score obtained:  0.46650553894745433


In [33]:
# Kernel SVM with RBF kernel (Hyperparameter: kernel parameter γ = 0)
model_5 =  svm.SVC(kernel="rbf")
tuning =  {"gamma":  np.linspace(0, 1, num=1)}
print("Model Description: ", model_5)
grid_cv5 = ms.GridSearchCV(model_5, tuning, scoring="accuracy", refit=True, cv=4, verbose=2, n_jobs=-1)
grid_cv5.fit(T1, Y1)
best_models_list.append(grid_cv5.best_estimator_)
print("---------------------------------------------")
print("Best Parameter obtained: ", tuning) 
print("Best Score obtained: ", grid_cv5.best_score_)

Model Description:  SVC()
Fitting 4 folds for each of 1 candidates, totalling 4 fits
---------------------------------------------
Best Parameter obtained:  {'gamma': array([0.])}
Best Score obtained:  0.5334944610525457


In [34]:
# KNN (Hyperparameter: number of neighbors)
model_6 = KNeighborsClassifier(n_jobs=-1)
tuning =  {"n_neighbors": list(range(2, 4))}
print("Model Description: ", model_6)
grid_cv6 = ms.GridSearchCV(model_6, tuning, scoring="accuracy", refit=True, cv=4, verbose=2, n_jobs=-1)
grid_cv6.fit(T1, Y1)
best_models_list.append(grid_cv6.best_estimator_)
print("---------------------------------------------")
print("Best Parameter obtained: ", tuning) 
print("Best Score obtained: ", grid_cv6.best_score_)

Model Description:  KNeighborsClassifier(n_jobs=-1)
Fitting 4 folds for each of 2 candidates, totalling 8 fits
---------------------------------------------
Best Parameter obtained:  {'n_neighbors': [2, 3]}
Best Score obtained:  0.7489452398850835


In [35]:
# Decision tree (Hyperparameter: min weight fraction leaf)

model_7 = DecisionTreeClassifier()
tuning =  {"min_weight_fraction_leaf": np.linspace(1e-10, 1, num=1)}
print("Model Description: ", model_7)
grid_cv7 = ms.GridSearchCV(model_7, tuning, scoring="accuracy", refit=True, cv=4, verbose=2, n_jobs=-1)
grid_cv7.fit(T1, Y1)
best_models_list.append(grid_cv7.best_estimator_)
print("---------------------------------------------")
print("Best Parameter obtained: ", tuning) 
print("Best Score obtained: ", grid_cv7.best_score_)

Model Description:  DecisionTreeClassifier()
Fitting 4 folds for each of 1 candidates, totalling 4 fits
---------------------------------------------
Best Parameter obtained:  {'min_weight_fraction_leaf': array([1.e-10])}
Best Score obtained:  0.8536494845661206


In [36]:
# Random forest (Hyperparameter: number of estimators)
model_8 = RandomForestClassifier(n_jobs=-1)
tuning =  {"n_estimators": list(range(100, 200, 100))}
print("Model Description: ", model_8)
grid_cv8 = ms.GridSearchCV(model_8, tuning, scoring="accuracy", refit=True, cv=4, verbose=2, n_jobs=-1)
grid_cv8.fit(T1, Y1)
best_models_list.append(grid_cv8.best_estimator_)
print("---------------------------------------------")
print("Best Parameter obtained: ", tuning) 
print("Best Score obtained: ", grid_cv8.best_score_)

Model Description:  RandomForestClassifier(n_jobs=-1)
Fitting 4 folds for each of 1 candidates, totalling 4 fits
---------------------------------------------
Best Parameter obtained:  {'n_estimators': [100]}
Best Score obtained:  0.8965020279365745


### Part e

In [37]:
# Tabulate the accuracy, precision, recall, specificity (-ve class) and senstivity (+ class) values
for models in best_models_list:
    print("Model Report: ", models)
    preds = models.predict(T2)
    print(metrics.classification_report(Y2, preds, labels=[-1, 1]))
    print(" -----------------------------------------------------------------------")
    print(" ")

Model Report:  LogisticRegression(C=1e-10, n_jobs=-1)
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       387
           1       0.53      1.00      0.70       442

    accuracy                           0.53       829
   macro avg       0.27      0.50      0.35       829
weighted avg       0.28      0.53      0.37       829

 -----------------------------------------------------------------------
 
Model Report:  LogisticRegression(C=1e-10, n_jobs=-1, penalty='l1', solver='saga')
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       387
           1       0.53      1.00      0.70       442

    accuracy                           0.53       829
   macro avg       0.27      0.50      0.35       829
weighted avg       0.28      0.53      0.37       829

 -----------------------------------------------------------------------
 
Model Report:  LinearSVC(C=1e-10)
              precision  

### Random forest perform best

### Part f

In [38]:
Logistic_L1 = best_models_list[1]
Logistic_L2 = best_models_list[0]
SVC_L1 = best_models_list[3]
SVC_L2 = best_models_list[2]

In [39]:
# Compare performance for L1 and L2 regularizer
print("Top 15 Weights for Logistic_L1 regularizer: \n", sorted(Logistic_L1.coef_.ravel(), reverse=True)[:15])
print("Top 15 Weights for Logistic_L2 regularizer: \n", sorted(Logistic_L2.coef_.ravel(), reverse=True)[:15])
print("-----------------------------------------------------")
print("Top 15 Weights for SVC_L1 regularizer: \n", sorted(SVC_L1.coef_.ravel(), reverse=True)[:15])
print("Top 15 Weights for SVC_L2 regularizer: \n", sorted(SVC_L2.coef_.ravel(), reverse=True)[:15])

Top 15 Weights for Logistic_L1 regularizer: 
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Top 15 Weights for Logistic_L2 regularizer: 
 [2.061734698193633e-08, 1.9031913485808018e-08, 1.8843590150276054e-08, 1.693361926941074e-08, 1.6672710831530927e-08, 1.572570810089231e-08, 1.565125117877032e-08, 1.5525015319405636e-08, 1.55105473122916e-08, 1.4913733641775458e-08, 1.4235854687861943e-08, 1.3043580028645431e-08, 1.2926881243306706e-08, 1.2888059807586866e-08, 1.2594101414912306e-08]
-----------------------------------------------------
Top 15 Weights for SVC_L1 regularizer: 
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Top 15 Weights for SVC_L2 regularizer: 
 [9.01997809994906e-08, 8.419974741127455e-08, 8.339977291512892e-08, 8.279979611950535e-08, 7.459974656993357e-08, 7.299982944818326e-08, 6.979979633983367e-08, 6.979978119198199e-08, 6.919982007469821e-08, 6.599980823153107e-08, 6.479983376166109e-08, 6.0799779834

### L1 regularizer making the weights zero for many parameters so it is not overfitting data and it is better regularizer.